In [ ]:
!apt-get update -y


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu b

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession
.builder
.appName("<app_name>")
.getOrCreate())

In [ ]:
spark.version

'3.1.2'

In [ ]:
! ls

sample_data


In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("sample").getOrCreate())

In [ ]:
import gdown

In [ ]:
gdown.download_folder(id='1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ', quiet=True, use_cookies=False)

['/content/chess/chess_schema.png',
 '/content/chess/chess_wc_history_game_info.csv',
 '/content/chess/chess_wc_history_moves.csv',
 '/content/chess/eco_codes.csv']

In [ ]:
df_games = spark.read.csv("/content/chess/chess_wc_history_game_info.csv", inferSchema=True, header=True)
df_moves= spark.read.csv("/content/chess/chess_wc_history_moves.csv", inferSchema=True, header=True)
df_codes= spark.read.csv("/content/chess/eco_codes.csv", inferSchema=True, header=True)

**1. List of Winners of Each World champions Trophy**

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

low = df_games.where((df_games.tournament_name.contains('WorldChamp')) & ~df_games.winner.isin(['draw']))
column = ['winner','tournament_name']

low = low.select(column).groupBy(column).count()

W_low = Window.partitionBy("tournament_name").orderBy(F.col('count').desc())

low = low.withColumn('countrowno',F.row_number().over(W_low)).where(F.col('countrowno') == 1)
column1 = ['count,countrowno']
low = low.orderBy(F.col('tournament_name')).drop(*column1)
low = low.select(column)
low.show()

+--------------------+---------------+
|              winner|tournament_name|
+--------------------+---------------+
|   Steinitz, William| WorldChamp1886|
|   Steinitz, William| WorldChamp1889|
|   Steinitz, William| WorldChamp1890|
|   Steinitz, William| WorldChamp1892|
|     Lasker, Emanuel| WorldChamp1894|
|     Lasker, Emanuel| WorldChamp1896|
|     Lasker, Emanuel| WorldChamp1907|
|     Lasker, Emanuel| WorldChamp1908|
|     Lasker, Emanuel| WorldChamp1909|
|    Schlechter, Carl|WorldChamp1910a|
|     Lasker, Emanuel|WorldChamp1910b|
|Capablanca, Jose ...| WorldChamp1921|
| Alekhine, Alexander| WorldChamp1927|
| Alekhine, Alexander| WorldChamp1929|
| Alekhine, Alexander| WorldChamp1934|
|           Euwe, Max| WorldChamp1935|
| Alekhine, Alexander| WorldChamp1937|
|  Botvinnik, Mikhail| WorldChamp1948|
|  Botvinnik, Mikhail| WorldChamp1951|
|  Botvinnik, Mikhail| WorldChamp1954|
+--------------------+---------------+
only showing top 20 rows



**Q2.List of Players with number of times they have won Tournament in descending order(Max to min)**

In [ ]:
from pyspark.sql.functions import col, asc,desc

lop = df_games.where(~df_games.winner.isin(['draw']))
lop = lop.select(['winner']).groupBy("winner").count()
print("---- List of Players with number of times they have won Tournament ----\n")
lop =lop.sort(col("count").desc())

#Rename Column
lop_n = lop.withColumnRenamed("winner","player_name") \
    .withColumnRenamed("count","number_of_wins")
lop_n.show()

---- List of Players with number of times they have won Tournament ----

+--------------------+--------------+
|         player_name|number_of_wins|
+--------------------+--------------+
|     Lasker, Emanuel|            52|
|     Karpov, Anatoly|            47|
|  Botvinnik, Mikhail|            46|
| Alekhine, Alexander|            43|
|   Steinitz, William|            43|
|           Topalov,V|            32|
|      Kasparov, Gary|            31|
|             Anand,V|            30|
|    Smyslov, Vassily|            24|
|            Adams,Mi|            20|
|      Kasimdzhanov,R|            19|
|          Grischuk,A|            17|
|           Svidler,P|            16|
|      Adams, Michael|            16|
|            Shirov,A|            15|
|            Bareev,E|            15|
|             Dreev,A|            15|
|   Chigorin, Mikhail|            14|
|Khalifman, Alexander|            14|
|           Euwe, Max|            14|
+--------------------+--------------+
only showing to

**Q3.Most and Least Popular eco move in world championship history.**

In [ ]:
eco = df_codes.select("eco","eco_name")
df_pop = eco.groupBy("eco_name").count()
df_pop = df_pop.sort(col("count").desc())
df_pop.registerTempTable("df_table")

spark.sql("select eco_name, count from df_table where count = (select max(count) from df_table) union all select eco_name, count from df_table where count = (select min(count) from df_table)").show()

+--------------------+-----+
|            eco_name|count|
+--------------------+-----+
|            Sicilian|   15|
|Sicilian, Acceler...|    1|
|       Three Knights|    1|
|Ruy Lopez, Bird's...|    1|
|King's Indian, Or...|    1|
|King's Indian Def...|    1|
|          Old Benoni|    1|
|    Sicilian, Alapin|    1|
|Sicilian, Dragon,...|    1|
|Dutch, Stonewall,...|    1|
|Queen's Gambit De...|    1|
|Catalan, Closed, ...|    1|
|King's Indian, Fo...|    1|
|Benoni, Classical...|    1|
|Sicilian, Richter...|    1|
|Sicilian, Fischer...|    1|
|King's Indian, Or...|    1|
|Benoni, Classical...|    1|
|   Caro-Kann Defense|    1|
|     French, Advance|    1|
+--------------------+-----+
only showing top 20 rows



**Q4.. Find the eco move with most winnings**

In [ ]:
df_ec = df_games.select("eco")
popular = df_ec.groupBy("eco").count()
popular = popular.sort(col("count").desc())

df_ec1 = df_codes.select("eco","eco_name")
popular.registerTempTable("df_table1")
df_ec1.registerTempTable("df_table2")

spark.sql("select t2.eco,t2.eco_name from df_table1 t1,df_table2 t2 where t1.count=(select max(count) from df_table1) and t1.eco=t2.eco").show()

+---+--------------+
|eco|      eco_name|
+---+--------------+
|C42|Petrov Defense|
+---+--------------+



**Q5.. Longest and shortest game ever played in a world championship in terms of move.**

In [ ]:
t1 = ["game_id","move_no_pair"]
t2 = ["game_id","tournament_name","event"]
 
df_t1 = df_moves.select(t1)
df_t2 = df_games.select(t2)

popular = df_t1.groupBy("game_id").count()
popular = popular.sort(col("count").desc())

popular.registerTempTable("df_table1")
df_t2.registerTempTable("df_table2")

spark.sql("select t2.game_id,t2.event,t2.tournament_name,t1.count as number_of_moves from df_table1 t1, df_table2 t2 where t1.count = (select max(count) from df_table1) and t1.game_id=t2.game_id union all select t2.game_id,t2.event,t2.tournament_name,t1.count as number_of_moves from df_table1 t1,df_table2 t2 where t1.count = (select min(count) from df_table1) and t1.game_id=t2.game_id").show()

+--------------------+--------------------+---------------+---------------+
|             game_id|               event|tournament_name|number_of_moves|
+--------------------+--------------------+---------------+---------------+
|4424a0a4-3732-407...|         FIDE WCh KO|  FideChamp2000|            291|
|1846cede-0037-4f0...|World Championshi...| WorldChamp1972|              1|
+--------------------+--------------------+---------------+---------------+



**Q6.. Shortest and Longest Draw game ever Played.**

In [ ]:
t1 = ["game_id","move_no_pair"]
t2 = ["game_id","tournament_name","event","winner"]
df_t1=df_moves.select(t1)
df_t2=df_games.select(t2).where(df_games.winner=="draw")

popular = df_t1.groupBy("game_id").count()
popular = popular.sort(col("count").desc())

popular.registerTempTable("df_table1")
df_t2.registerTempTable("df_table2")

spark.sql("select t2.game_id,t2.event,t2.tournament_name,t1.count as number_of_moves from df_table1 t1, df_table2 t2 where t1.count = (select max(count) from df_table1 t1, df_table2 t2 where t2.game_id=t1.game_id) and t1.game_id=t2.game_id union all select t2.game_id,t2.event,t2.tournament_name,t1.count as number_of_moves from df_table1 t1, df_table2 t2 where t1.count = (select min(count) from df_table1 t1, df_table2 t2 where t2.game_id=t1.game_id) and t1.game_id=t2.game_id").show()

+--------------------+-------------+---------------+---------------+
|             game_id|        event|tournament_name|number_of_moves|
+--------------------+-------------+---------------+---------------+
|88f34084-e4df-490...|  FIDE WCh KO|  FideChamp2002|            258|
|a27c2b95-fb62-4b6...|FIDE-Wch k.o.|  FideChamp1998|             12|
+--------------------+-------------+---------------+---------------+



**Q7.. Shortest and Longest Draw game ever Played.**

In [ ]:
most = df_games.select("winner","winner_elo").where((df_games.winner!="null") & (df_games.winner!="draw")).distinct()
most_rated = most.sort(col("winner_elo").desc())
most_rated.registerTempTable("df_table1")

spark.sql("select winner as player_name, winner_elo as elo from df_table1 where winner_elo = (select max(winner_elo) from df_table1) union all select winner as player_name, winner_elo as elo from df_table1 where winner_elo = (select min(winner_elo) from df_table1)").show()

+---------------+----+
|    player_name| elo|
+---------------+----+
|Carlsen, Magnus|2870|
|     Ciuksyte,D|2271|
+---------------+----+



**Q8.3rd Last Player with most Loss**

In [ ]:
df_lose = df_games.select("loser").where((df_games.loser!="null") & (df_games.loser!="draw"))
df_lose = df_lose.groupBy("loser").count()
df_lose = df_lose.sort(col("count").desc())
df_lose.registerTempTable("df_table")
spark.sql("select loser from df_table limit 1").show()

+-----------------+
|            loser|
+-----------------+
|Steinitz, William|
+-----------------+

